# Prompting Guide for Cohere Command R and Command R+ Models on Amazon Bedrock
---
## Introduction
This notebook gives an overview of prompting with Cohere Command R and R+ models on Amazon Bedrock.This notebook will leverage the Amazon Bedrock APIs.

Throughout the notebook, we highlight different prompting technqiues such as more familiar methods like few-shot prompting as well as leveraging steps and context to improve the models outputs and response. 

---

## Amazon Bedrock
Amazon Bedrock is a fully managed service provided by Amazon Web Services (AWS) that facilitates the use of high-performing foundation models (FMs) from leading AI companies and Amazon itself. It is designed to help developers build and deploy generative AI applications efficiently and securely without the need to manage infrastructure.

Leveragin Cohere's models on Amazon Bedrock can supports a wide range of applications. Some of them include:

- Text generation (e.g., stories, essays, emails)
- Chatbots and virtual assistants
- Search and information synthesis
- Text summarization


Amazon Bedrock simplifies the development and deployment of generative AI applications by providing access to top foundation models, enabling private customization, and offering a serverless, secure, and cost-effective environment for AI innovation.

---

## Cohere Command R and Command R+ models

There are multiple LLMs from Cohere supported on Amazon Bedrock today and these include:
1. #### Cohere Command R+
    - **Description:** Command R+ is Cohere's most powerful generative language model optimized for long-context tasks, such as retrieval-augmented generation (RAG) and multi-step tool use.
    - **Max Tokens:** 128K
    - **Languages:** English, French, Spanish, Italian, German, Portuguese, Japanese, Korean, Arabic, and Chinese
    - **Supported Use Cases:** Text generation, text summarization, chat, knowledge assistants, Q&A, RAG.

2. #### Cohere Command R
    - **Description:** Command R is Cohere's generative language model optimized for long-context tasks, such as retrieval-augmented generation (RAG) and tools, and large scale production workloads.
    - **Max Tokens:** 128K
    - **Languages:** English, French, Spanish, Italian, German, Portuguese, Japanese, Korean, Arabic, and Chinese
    - **Supported Use Cases:** Text generation, text summarization, chat, knowledge assistants, Q&A, RAG.

3. #### Cohere Command
    - **Description:** Command is Cohere’s generative large language model (LLM).
    - **Max Tokens:** 4K
    - **Languages:** English
    - **Supported Use Cases:** Chat, text generation, text summarization.

4. #### Cohere Command Light
    - **Description:** Command Light is a smaller version of Command, Cohere's generative LLM.
    - **Max Tokens:** 4K
    - **Languages:** English
    - **Supported Use Cases:** Chat, text generation, text summarization.

These are the LLMs available from Cohere on Amazon Bedrock. Also, there are two embeddings models supported on Amazon Bedrock: 
1/ Cohere Embed - English and 
2/ Cohere Embed - Multilingual.

The overall Command R family of models are highly scalable language models that balance high performance with strong accuracy. Command R and Command R+ are a part of the Command R family and are optimized for RAG based workflows such as conversational interaction and long context tasks, enabling companies to move beyond proof of concept and into production. These powerful models are designed to handle complex tasks with high performance and strong accuracy, making them suitable for real-world applications.

Command R boasts high precision on RAG and tool use tasks, low latency and high throughput, a long 128,000-token context length, and strong capabilities across 10 key languages: English, French, Spanish, Italian, German, Portuguese, Japanese, Korean, Arabic, and Chinese.

Command R+ is the newest model, optimized for extremely performant conversational interaction and long-context tasks. It is recommended for workflows that lean on complex RAG functionality and multi-step tool use (agents), while Cohere R is well-suited for simpler RAG and single-step tool use tasks, as well as applications where price is a major consideration. The model is optimized to perform well in the following languages: English, French, Spanish, Italian, German, Brazilian Portuguese, Japanese, Korean, Simplified Chinese, and Arabic.

---

## Prerequisites 

<b>Follow the steps below if you are running the notebook in your owned AWS enviornment.<br>
    If running through the workshop studio, SKIP the steps below<b>
    
1. Ensure you have requested access to the models provided by Cohere in the Bedrock console by clicking "model access." Instructions can be found here: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html
2. Make sure you have the permissions to access Bedrock and you have the correct IAM permissions from your administrator 



---

## Getting Started

To get started using Cohere models on Bedrock we need to install dependencies. Here, we will install all the required dependencies to run this notebook.

In [ ]:
%pip install boto3 IPython --quiet

Now lets import the required modules to run the notbook and set up the Bedrock client

In [ ]:
import boto3
import json
import time

### Select Model

Below we give choice of using Command R + models and Command R for the rest of our notebook. By default we will select Command R + and we also create the Bedrock client "bedrock_rt".

In [ ]:
DEFAULT_MODEL= "cohere.command-r-plus-v1:0"
COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
COMMAND_R = "cohere.command-r-v1:0"
model_id = DEFAULT_MODEL

Now lets import the required modules to run the notbook and set up the Bedrock client

In [ ]:
bedrock_rt = boto3.client(service_name="bedrock-runtime")

---
### Supported parameters 

The Cohere Command models have the following inference parameters available today. The one that is required is "message" parameter where the others are able to be added in a call to Bedrock where needed:

```
{
    "message": string,
    "chat_history": [
        {
            "role":"USER or CHATBOT",
            "message": string
        }
    ],
    "documents": [
        {"title": string, "snippet": string},
    ],
    "search_queries_only" : boolean,
    "preamble" : string,
    "max_tokens": int,
    "temperature": float,
    "p": float,
    "k": float,
    "prompt_truncation" : string,
    "frequency_penalty" : float,
    "presence_penalty" : float,
    "seed" : int,
    "return_prompt" : boolean,
    "tools" : [
        {
            "name": string,
            "description": string,
            "parameter_definitions": {
                "parameter name": {
                    "description": string,
                    "type": string,
                    "required": boolean
                }
            }
        }
    ],
    "tool_results" : [
        {
            "call": {
                "name": string,
                "parameters": {
                "parameter name": string
                }
            },
        "outputs": [
                {
                "text": string
                }
            ]
        }
    ],
    "stop_sequences": [string],
    "raw_prompting" : boolean

}
```


There are quite a few inference parameters available for you to use on Bedrock for Cohere Command R model family specifically. In this notebook we wille explore creating an example function for how to call the invoke_model API as well as how to use a few of the above parameters.

In [ ]:
#a function to generate the text
#temp set to 0.3 by default
def generate_text(prompt, model_id, temp=0.3):
    body = {
    'message': prompt,
    'temperature': temp,
    'preamble':""
    }
# Invoke the Bedrock model
    response = bedrock_rt.invoke_model_with_response_stream(
        modelId= model_id,
        body=json.dumps(body)
    )
# Print the response
    stream = response.get('body')
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                byte = chunk.get('bytes').decode()
                output=json.loads(byte)
            if output['event_type'] == 'text-generation':
                print(output['text'], end='')

---

### Context
The most effective prompts are those that are clear, concise, specific, and include examples of exactly what a response should look like.Providing context and delimiters are a few methods to the most effective responses from the Command family of models. 
Let's see an example of different outputs when we give context vs when we don't.

#### Without Context

In [ ]:
user_input ="Give financial highlights from Amazon's earnings call"
prompt = user_input
response = generate_text(prompt, model_id)

#### With Context

In [ ]:
context = """Think back to the last time you were a bot helping summarize financial reports for a company to present.

Worldwide revenue was $143.3 billion, representing a 13% increase year-over-year, excluding the impact of foreign exchange and near the top end of our guidance range. I'd like to highlight a couple of points to help you interpret our growth rates.

First, we saw an impact from leap year in Q1, which added approximately 120 basis points to the year-over-year quarterly revenue growth rate. Second, while I typically talk about growth rates, excluding the impact of year-over-year changes in foreign exchange, we did see an unfavorable impact from global currencies weakening against the US dollar, more than we had planned in Q1. This led to a $700 million or 50 basis point headwind to revenue relative to what we guided. Excluding this FX headwind, we would have exceeded the top end of our guidance range.

Worldwide operating income was $15.3 billion, which was our highest quarterly income ever, and it was $3.3 billion above the high end of our guidance range. This was driven by strong operational performance across all three reportable segments and better-than-expected operating leverage, including lower cost to serve. The impact on operating income from our Q1 FX rate headwind was negligible. I'll speak more to our profitability trends in a moment.

In the North America segment, first quarter revenue was $86.3 billion, an increase of 12% year-over-year. In the international segment, revenue was $31.9 billion, an increase of 11% year-over-year, excluding the impact of foreign exchange. We remain focused on the inputs that matter most to our customers: selection, price, and convenience.

During the quarter, around the world, we help customers save with our shopping events. We added selection, including premium and luxury brands, and we delivered our fastest speeds ever for Prime members. Third-party sellers continue to be an important part of our offering. Third-party seller services revenue increased 16% year-over-year, excluding the impact of foreign exchange. We saw strong 3P unit growth, coupled with increased adoption of our optional services, such as fulfillment and global logistics. For the quarter, third-party seller unit mix was 61%, up 200 basis points year-over-year.

Shifting to profitability, North America segment operating income was $5 billion, an increase of $4.1 billion year-over-year. Operating margin was 5.8%, up 460 basis points year-over-year. We saw improvements in our cost to serve, including continued benefit from our work to regionalize our operations, savings from more consolidated customer shipments, and improved leverage driven by strong unit growth and lower transportation rates.

In our international segment, operating income was $903 million, an improvement of $2.2 billion year-over-year. Operating margin was 2.8%, up 710 basis points year-over-year. This is primarily driven by our established countries as we improve cost efficiencies through network design enhancements and improved volume leverage. Additionally, we saw good progress in our emerging countries as they expand their customer offerings and make strides on their respective journeys to profitability.

Looking ahead, we see several opportunities to further lower cost to serve and improved profitability in our worldwide stores business while still investing to improve the customer experience. Within our fulfillment network, we are focused on investing in our inbound network, streamlining and standardizing process paths, and adding robotics and automation. These improvement opportunities will take time. However, we have a solid plan in place and we like the path we're on.

Advertising remains an important contributor to profitability in North America and international segments. We see many opportunities to grow our offerings, both in the areas that are driving growth today like sponsored products and in areas that are newer, like streaming TV ads.

Moving to AWS. Revenue was $25 billion, an increase of 17% year-over-year, and AWS is now a $100 billion annualized revenue run rate business. Excluding the impact from leap year, AWS revenue increased approximately 16% year-over-year.

During the first quarter, we saw growth in both generative AI and non-generative AI workloads across a diverse group of customers and across industries as companies are shifting their focus towards driving innovation and bringing new workloads to the cloud. Additionally, we continue to see the impact of cost optimizations diminish. While there always be a level of ongoing optimization, we think the majority of the recent cycle is behind us, and we're likely closer to a steady state of these optimization efforts.

AWS operating income was $9.4 billion, an increase of $4.3 billion year-over-year. As a reminder, these results include the impact from the change in the estimated useful life of our servers, which primarily benefits the AWS segment. We made progress in managing our infrastructure and fixed costs while still growing at a healthy rate, which has resulted in improved leverage.

As we've said in the past, over time, we expect the AWS operating margins to fluctuate, driven in part by the level of investments we are making in the business. We remain focused on driving efficiencies across the business, which enables us to invest to support the strong growth we're seeing in AWS, including generative AI, which brings us to capital investments. As a reminder, we define these as the combination of CapEx plus equipment finance leases.

In 2023, overall capital investments were $48.4 billion. As I mentioned, we're seeing strong AWS demand in both generative AI and our non-generative AI workloads, with customers signing up for longer deals, making bigger commitments. Still relatively early days in generative AI and more broadly, the cloud space, and we see sizable opportunity for growth.

We anticipate our overall capital expenditures to meaningfully increase year-over-year in 2024, primarily driven by higher infrastructure CapEx to support growth in AWS, including generative AI.


"""

user_input = "Give financial highlights from Amazon's earnings call"

prompt = f"""{context}
Given the information above, answer this question: {user_input}"""

response = generate_text(prompt, model_id)

You may have noticed a difference in the outputs. This is for a few reasons but the main being that the first example didn't use any context so the model did not know which earnings call to generate highlights for. 

We did not give the LLM the transcript exerpt that we wanted it to pull from, thus the first example did not give the information we needed. As compared to the second example with context, the model is able to understand more of the actual data you'd like to use and this helps provide more accurate answers. The first example did not know which earnings report to pull from, where as the second ask was including specific items from the transcript from the most recent earnings report from Amazon. 

There are multiple ways to add even more context and also prevent additional hallucinations. In theory we would not want to pass the entire transcript to the LLM. This is because we want to optimize how many tokens we send into the Bedrock API for cost and performance reasons. This also helps the LLM generate more accurate responses when we use prompt strategies and techniques like Retrieval Augment Generation(RAG) that we will dive into later.

---

### Prompting Techniques
Cohere developed core guidance for advanced prompting techniques for their models: https://docs.cohere.com/docs/advanced-prompt-engineering-techniques#few-shot-prompting. 
The following examples will step through few shot prompting and chain of thought prompting examples for Cohere Command models. 

#### Few-Shot Prompting
- Few-shot prompting involves providing the model with a small number of examples demonstrating the desired task before asking it to perform that task on new inputs.
- Few-shot prompting is useful when you want the model to learn and generalize a specific task or skill from a small number of examples, rather than relying solely on its pre-training.
- This technique is also useful for when the task requires understanding a particular format, structure, or pattern that can be effectively conveyed through examples.
You need the model's outputs to closely match the style, tone, or conventions demonstrated in the examples.


#### Delimiters
In the below example you will also see the use of the delimiter '##'. Providing a well-structured and unambiguous prompt can enhance the performance of a large language model (LLM). It is beneficial to place the instructions at the start of the prompt and delineate different sections, such as instructions, context, and resources, using descriptive headers. These headers can be made more prominent by prefixing them with '##'. You can see in the below example that '## Examples' was used to show the start of the examples.


Let's see the below example for how you can use few-shot prompting to create outreach emails based on customer sentiment.

In [ ]:
examples = """
## Examples
Customer Sentiment: The customer had a negative experience with the product. They found it difficult to use and felt that the instructions were unclear.
Marketing Outreach Email:
Subject: We're sorry for the inconvenience

Dear [Customer Name],

We're sorry to hear that you had a negative experience with our product. At [Company Name], we strive to provide our customers with high-quality products and exceptional service.

We understand that the instructions may have been unclear, and we apologize for any frustration this may have caused. We value your feedback and would like to learn more about your experience to improve our product and documentation.

As a token of our appreciation for your patience and understanding, we'd like to offer you a 20% discount on your next purchase. Please use the code FEEDBACK20 at checkout.

If you have any further questions or concerns, please don't hesitate to reach out to our customer support team. We're here to help and ensure that you have a positive experience with our products.

Thank you for your continued support.

Best regards,
[Your Name]
[Company Name]

Customer Sentiment: The customer had a mixed experience with the product. They found some features useful but felt that others were lacking or confusing.
Marketing Outreach Email:
Subject: We value your feedback

Dear [Customer Name],

Thank you for sharing your experience with our product. We're glad to hear that you found some features useful, but we apologize for any confusion or frustration caused by the lacking or confusing aspects.

At [Company Name], we're committed to continuously improving our products and services based on customer feedback. Your input is invaluable in helping us identify areas for improvement and better meet your needs.

As a token of our appreciation for your feedback, we'd like to offer you a 15% discount on your next purchase. Please use the code VALUED15 at checkout.

We'd also like to invite you to schedule a call with one of our product specialists to discuss your experience in more detail. This will help us better understand your specific needs and concerns, and ensure that future updates address them effectively.

Thank you for choosing [Company Name], and we look forward to continuing to serve you in the future.

Best regards,
[Your Name]
[Company Name]

Customer Sentiment: The customer was frustrated with the product and felt that it did not meet their expectations. They had issues with the setup process and found the user interface confusing.
Marketing Outreach Email:
Subject: We're here to help

Dear [Customer Name],

We're sorry to hear that our product did not meet your expectations. At [Company Name], we take customer satisfaction seriously, and we want to ensure that you have a positive experience with our products and services.

We understand that the setup process and user interface caused frustration, and we apologize for any inconvenience this may have caused. We value your feedback and would like to learn more about your specific issues to improve our product and documentation.

As a token of our appreciation for your patience and understanding, we'd like to offer you a 25% discount on your next purchase. Please use the code SUPPORT25 at checkout.

Additionally, we'd like to invite you to schedule a one-on-one session with one of our product experts. During this session, they can walk you through the setup process, address any concerns you may have, and provide personalized guidance to help you get the most out of our product.

At [Company Name], we're committed to providing exceptional customer service, and we want to ensure that you have a positive experience moving forward.

Thank you for choosing [Company Name], and we look forward to addressing your concerns and exceeding your expectations.

Best regards,
[Your Name]
[Company Name]

"""

user_input = "The customer was generally satisfied with the product but felt that the pricing was too high for the features offered."

prompt = f""" ## Below there are a few examples of the creationg of marketing outreach email with an email subject based on a customer's sentiment. Do not include anything additional. Please generate an email based on user input following
the below examples {examples} to answer this question: {user_input}"""

response = generate_text(prompt, model_id)

---

### Chain of Thought Prompting (COT)
To steer the model toward generating higher-quality responses, it can be helpful to add instructions for the model to generate intermediate steps before generating the final output. The information generated during these steps helps enrich the model’s context before it generates the final response.

#### Without Steps

In [ ]:
user_input = "technology"

prompt = f"""Generate a startup idea for this industry: {user_input}"""

response = generate_text(prompt, model_id)

#### With Steps

In [ ]:
user_input = "technology"

prompt = f"""Generate a startup idea for this industry: {user_input}
First, describe the problem to be solved.
Next, describe the target audience of this startup idea.
Next, describe the startup idea and how it solves the problem for the target audience.
Next, provide a name for the given startup.

Use the following format:
Industry: <the given industry>
The Problem: <the given problem>
Audience: <the given target audience>
Startup Idea: <the given idea>
Startup Name: <the given name>"""

response = generate_text(prompt, model_id)

---

### RAG Examples

 Cohere's Command R family of models targets the emerging “scalable” category of models that balance high efficiency with strong accuracy, enabling companies to move beyond proof of concept, and into production. Command R is a generative model optimized for long context tasks such as retrieval-augmented generation (RAG). Command R+ is ideal for advanced Retrieval Augmented Generation (RAG) and also provides citation to reduce hallucinations. You may be wondering, What is RAG? Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts. Let's see an example below of this using the Bedrock API when we create a new function for helping us decide which AWS service to use:

In [ ]:
#function to generate text leveraging hard coded documents
def generate_RAG(prompt, model_id, temp):
    body = {
    'message': prompt,
    'temperature': temp,
    'preamble':"",
    'documents': [
    {
       "id": "s3_page",
       "title": "S3 product page",
       "snippet": "Amazon S3 is an object storage service offering industry-leading scalability, data availability, security, and performance",
       "url": "https://aws.amazon.com/s3/faqs/?nc=sn&loc=7",
    },
    {
        "id": "ec2_page",
        "title": "EC2 product page",
        "snippet": "Amazon EC2 is a web service that provides resizable compute capacity in the cloud. It is designed to make web-scale computing easier for developers.",
        "url": "https://aws.amazon.com/ec2/faqs/",
    }, 
    {
        "id": "dynamodb_page",
        "title": "DynamoDB product page",
        "snippet": "Amazon DynamoDB is a fully managed NoSQL database service that makes it easy to build and scale a real-time application.",
        "url": "https://aws.amazon.com/dynamodb/faqs/?refid=9eb02e4d-80e0-4f27-a621-b90b3c870bf3",
    }
    ]
    }
# Invoke the Bedrock model
    response = bedrock_rt.invoke_model(
        modelId= model_id,
        body=json.dumps(body)
    )
    response_body = json.loads(response.get('body').read())
    # Print the response  
    responses = []
    responses = [response_body['text'], response_body['documents']]
    return responses

In [ ]:
prompt = "What service should I use to store PDFs?"

response = generate_RAG(prompt, model_id, 0.3)

#print the response and the citations that are returned from Bedrock API
print(response[0], "\n\n", "Citations", response[1], sep="")

In the above example, we only gave a high level description of S3 in the snippet but when we use the "URL" key, we are able to give more information. Then, the command R+ model takes care of the RAG approach for us since the model has been trained specifically for these types of use cases. In the Bedrock response, we can also return "response_body['documents]" which gives a high level citation for what the model referenced when giving the response.

---

## Command R+ vs Command R

Both of these models are a part of Cohere's newest model family and while they have similar capabilites, there are areas where they differ in terms of performance and responses

### Cohere Command R Model

**Capabilities:**

- Generates coherent and contextually relevant text based on input prompts.
- Proficient in understanding and processing natural language, making it suitable for tasks like summarization, translation, and question answering.
- Can be fine-tuned with specific datasets to better align with particular use cases or industry-specific language.
    Retrieval-Augmented Generation (RAG): Supports RAG, which allows the model to retrieve relevant documents or information from a database to enhance the quality and accuracy of generated responses.

**Performance:**

- Designed to be efficient in generating text with a balance between speed and quality.
- Provides accurate and contextually relevant responses for a wide range of natural language processing tasks.
- Can be scaled to handle large volumes of requests, making it suitable for enterprise applications.

### Cohere Command R+ Model
**Capabilities:**

- Builds on the capabilities of the Command R model with improved performance in generating high-quality, contextually accurate text.
- Offers superior understanding and processing of complex language structures, making it more effective for nuanced tasks.
- Provides more advanced fine-tuning options, allowing for deeper customization to meet specific needs.
- Capable of handling longer context windows, which is beneficial for tasks requiring extensive background information.
- Enhanced retrieval-augmented generation capabilities, allowing for more accurate and contextually relevant information retrieval and integration into responses.

**Performance:**

- Offers improved performance over the Command R model, with higher accuracy and better handling of complex language tasks.
- Maintains efficiency while providing faster response times, even with more complex queries.
- Enhanced scalability to support even larger volumes of requests and more demanding applications.
- More robust in handling diverse and challenging language tasks, providing reliable performance across different use cases.

### Observations
The Cohere Command R+ model is more performant than the Command R model due to several key enhancements:

- **Improved Text Generation:** The Command R+ model is optimized to generate higher-quality text that is more contextually accurate and coherent. For example, we observed Command R+ model provides more specific data points and a slightly more detailed responses with metrics.
- **Advanced Language Understanding:** With superior language understanding capabilities, the Command R+ model can process and interpret more complex language structures, leading to more accurate and nuanced responses.
- **Extended Context Handling:** The ability to handle longer context windows allows the Command R+ model to incorporate more background information into its responses, improving the relevance and accuracy of the generated text.
- **Enhanced RAG Capabilities:** The improved retrieval-augmented generation capabilities of the Command R+ model enable it to retrieve and integrate more accurate and contextually relevant information from external sources, enhancing the overall quality of the responses. We observed that this model can detect which items will clearly answer a user query and are able to make that decision based on the data or documents given.
- **Efficiency and Speed:** Despite the increased capabilities, the Command R+ model may have slightly longer response times due to its more complex processing.

These enhancements make the Cohere Command R+ model a more powerful and versatile tool for a wide range of natural language processing tasks, providing better performance and more accurate results compared to the Command R model. Let's see an example showing differences in output for the same task.


### Examples

The following prompts give examples show casing the difference in outputs between Command R and Command R+ when it comes to text generation and RAG capabilities. The responses from Command R lack the detail that Command R+ is able to provide, but it has a lower runtime hence being more efficient. The Command R+ model is able to provide more enhanced details and is aware of other contextual data to enrich the response

In [ ]:
#local function to measure the runtime of each model we will call
def measure_runtime(func, prompt, model_id, temp):
    start_time = time.time()
    result = func(prompt, model_id, temp)
    end_time = time.time()
    runtime = end_time - start_time
    return result, runtime


context = """
XYZ Corp's Q2 2024 Financial Report:
- Revenue: $100 million, representing a 10% increase year-over-year. This growth was primarily driven by strong sales in the North American and European markets.
- Net Profit: $15 million, up 5% year-over-year. The increase in net profit was attributed to improved operational efficiencies and cost-cutting measures.
- Debt-to-Equity Ratio: 0.8, down from 1.0 last year. The reduction in the debt-to-equity ratio indicates a stronger balance sheet and reduced financial leverage.
- Cash Reserves: $30 million, a 20% increase year-over-year. The significant increase in cash reserves provides the company with greater financial flexibility and the ability to invest in future growth opportunities.
- New Product Launches: The company successfully launched three new products this quarter, contributing to the overall revenue growth.
- Market Expansion: XYZ Corp expanded its market presence in Asia, which is expected to drive future growth.

Overall, XYZ Corp has demonstrated solid financial performance this quarter, with notable improvements in revenue, net profit, and cash reserves. 
"""

prompt = f""" 
## Context
{context}
## Question
Summarize the company's financial health and provide a brief recommendation for potential investors
"""


# Generate responses and measure runtime
response_command_r, runtime_command_r = measure_runtime(generate_text, prompt, COMMAND_R, temp=0.2)
# Print the responses side by side
print("Response from Command R Model:")
print(response_command_r)
# Print the runtimes
print(f"\nRuntime for Command R Model: {runtime_command_r:.4f} seconds")

print("\n")
response_command_r_plus, runtime_command_r_plus = measure_runtime(generate_text, prompt, COMMAND_R_PLUS, temp=0.2)
print("\nResponse from Command R+ Model:")
print(response_command_r_plus)
print(f"\nRuntime for Command R+ Model: {runtime_command_r_plus:.4f} seconds")

In [ ]:
#Code to show enhanced RAG using predefined function generate_RAG() used earlier

prompt = "What service should I use to store my documents?"

# Generate responses and measure runtime
response_command_r, runtime_command_r = measure_runtime(generate_RAG, prompt, COMMAND_R, temp=0.3)
# Print the responses side by side
print("Response from Command R Model:")
print(response_command_r[0], "\n\n", "Citations", response_command_r[1], sep="")
# Print the runtimes
print(f"\nRuntime for Command R Model: {runtime_command_r:.4f} seconds")
print("\n")
response_command_r_plus, runtime_command_r_plus = measure_runtime(generate_RAG, prompt, COMMAND_R_PLUS, temp=0.3)
print("\nResponse from Command R+ Model:")
print(response_command_r_plus[0], "\n\n", "Citations", response_command_r_plus[1], sep="")
print(f"\nRuntime for Command R+ Model: {runtime_command_r_plus:.4f} seconds")



---

## Conclusion

In this notebook, we discussed examples of different prompting techniques when using the Cohere Command R and R+ models. Advanced prompting techniques like proving context, including delimiters, Chain of Thought Prompting can be explored to develop even stronger prompts per use case. We explored different base level techniques for Cohere's models that are similar to other general prompting techniques for other LLMs and models from other 3P model providers available on Bedrock today. 

Finally, we discussed using RAG based approach with Bedrock APIs. It’s worth noting that the prompting techniques discussed, including RAG, does not guarantee accuracy. We explored how providing context to the model is important to inform the model on how to reply but if the documents, data or even prompt information is out of date, the LLM will respond inaccurately. The processes discussed above help provide low lift ways to reduce risk and even with the feature of providing citations helps with the explainability of the model's response.